In [1]:
import glob
import json
import numpy as np

In [2]:
years = glob.glob('nba_lineup_data/*')
lineups = []
y = None
for year in years:
    year_dict = json.load(open(year,'r'))
    for k, v in year_dict.items():
        lineups.append((k,v))

In [3]:
player_to_idx = {}
idx_to_player = []
normal_name_to_idx = {}
i = 0
for lineup in lineups:
    l = eval(lineup[0])
    o_lineup = l[0]
    d_lineup = l[1]
    for player in o_lineup + d_lineup:
        if player not in player_to_idx:
            player_to_idx[player] = i
            normal_name_to_idx[player.split('*')[0].lower()] = i
            idx_to_player.append(player)
            i += 1

In [30]:
all_stats = ['PTS','TS%','FGA','AST','REB','STL','BLK','2PA','2PM','3PA','3PM','FTA','FTM']
def get_normalized_team_stats(lineup,info):
    box = info['box']
    l = eval(lineup)
    o_lineup = [player_to_idx[p] for p in l[0]]
    d_lineup = [player_to_idx[p] for p in l[1]]
    num = info['num']
    meta = (num, info['score']/num)
    statsheet = np.zeros((10,13))
    for i, player in enumerate((l[0]+l[1])[:10]):
        for j, stat in enumerate(all_stats):
            if player in box and stat in box[player]:
                statsheet[i][j] = box[player][stat] * 1.0 / num
        #TS% = 0.5 * pts / (FGA + .44 * FTA)
        denom = (statsheet[i][2] + .44 * statsheet[i][11])
        if denom > 0:
            statsheet[i][1] = 0.5 * statsheet[i][0] / denom
            
    return o_lineup, d_lineup, statsheet, meta

([161, 330, 35, 257, 64], [184, 353, 136, 218, 137], array([[0.4 , 0.5 , 0.4 , 0.  , 0.  , 0.  , 0.  , 0.4 , 0.2 , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.6 , 1.5 , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.2 ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [1.  , 1.25, 0.4 , 0.4 , 0.  , 0.  , 0.  , 0.2 , 0.2 , 0.2 , 0.2 ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  

In [31]:
lineups = [get_normalized_team_stats(l[0],l[1]) for l in lineups]

In [35]:
import pickle
with open("processed_data/pickled_lineups1998-2018",'wb') as outfile:
    pickle.dump(lineups,outfile)

In [39]:
with open("processed_data/lineup_metadata1998-2018",'wb') as outfile:
    pickle.dump((player_to_idx, idx_to_player, normal_name_to_idx, all_stats), outfile)